参考链接：https://blog.csdn.net/qq_45677854/article/details/117401939

BLEU全称是：a Method for Automatic Evaluation of Machine Translation(是一种用来评估机器翻译的评价指标)广泛出现在，文本生成的论文当中。
BLEU采用一种N-Gram的匹配规则，具体来说就是比较译文和参考文献之间的N组词的相似度。

In [1]:
import numpy as np
from collections import Counter

In [16]:
def BLEU(references,generated,max_grams=4,weights=None):
    """
    references：参考译文列表，包含多个参考译文，用于计算BLEU得分。
    generated：生成译文，待评估的译文。
    max_grams：最大匹配词元数，即n-gram的最大n值，默认为4。
    weights：权重，用于加权计算不同词元数的匹配率，默认为None。
    """

    # 将参考译文和生成译文转换成小写并按空格分割成列表
    ref_list=[ref.lower().split(" ") for ref in references]
    gen=generated.lower().split(" ")
    cpn = np.empty((max_grams,), dtype=np.float32)  # 创建一个空数组来存储不同词元数的匹配率
    for n in range(1, max_grams + 1):
        gen_gram = [" ".join(gen[i:i+n]) for i in range(0, len(gen) - n + 1)]  # 生成译文的 n-gram 列表
        refs_gram = [[" ".join(ref[i:i+n]) for i in range(0, len(ref) - n + 1)] for ref in ref_list]  # 参考译文的 n-gram 列表
        g_counter = Counter(gen_gram)  # 统计生成译文中每个 n-gram 的频次
        r_counters = [Counter(ref_gram) for ref_gram in refs_gram]  # 统计每个参考译文中每个 n-gram 的频次
        count_clip = 0
        for k, v in g_counter.items():
            count_clip += min(v, max([r.get(k, 0) for r in r_counters]))  # 计算生成译文与参考译文中共有的 n-gram 的频次
        cpn[n-1] = count_clip / sum(g_counter.values())  # 计算词元匹配率

    ls = len(gen)  # 生成译文的长度
    lc = max([len(ref) for ref in ref_list])  # 参考译文中最长的长度
    brevity_penalty = 1 if lc > ls else np.exp(1 - ls / lc)  # 计算长度惩罚因子

    if weights is None:
        weights = np.ones_like(cpn)  # 如果未提供权重，则初始化为全1的数组
    bleu = brevity_penalty * np.exp(np.mean(weights * np.log(cpn)))  # 计算BLEU得分

    print(cpn)
    print(gen)
    print(ref_list)
    return bleu

函数首先将参考译文和生成译文进行预处理，将它们转换为小写并按空格分割成词元列表。然后，函数计算不同词元数的匹配率，即生成译文中的n-gram在参考译文中的出现频次的最大值，除以生成译文中的n-gram总数，得到匹配率列表cpn。接下来，函数计算长度惩罚因子，根据生成译文的长度和参考译文中最长的长度计算得到。最后，如果未提供权重，则将权重初始化为全1的数组，然后使用加权的匹配率计算BLEU得分。

函数的返回值是计算得到的BLEU得分。

In [17]:
bleu = BLEU(["The cat is on the mat", "There is a cat on the mat"], "A cat is on the mat", 3)
print(bleu)

[1.   1.   0.75]
['a', 'cat', 'is', 'on', 'the', 'mat']
[['the', 'cat', 'is', 'on', 'the', 'mat'], ['there', 'is', 'a', 'cat', 'on', 'the', 'mat']]
0.9085603356361389
